In [ ]:
import sys
import os
import h5py
import json
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

from stimuli_f0_labels import get_f0_bins, f0_to_label

fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/processed/dataset_2019-11-22-2300/PND_sr32000_v08.hdf5'
# fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/processed/dataset_2019-11-16-2300/PND_sr32000_v07.hdf5'
# fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/processed/dataset_2019-08-16-1200/PND_sr32000_v04.hdf5'

f = h5py.File(fn, 'r')
for v in f.values():
    print(v)
for v in f['config'].values():
    print(v)

file_indexes = f['source_file_index'][:]
segment_indexes = f['source_file_row'][:]
f0_values = f['nopad_f0_mean'][:]
source_file_encoding_dict = f['config/source_file_encoding_dict'][0]
source_file_encoding_dict = source_file_encoding_dict.replace('"', '"""')
source_file_encoding_dict = source_file_encoding_dict.replace('\'', '"')
source_file_encoding_dict = json.loads(source_file_encoding_dict)

f.close()


In [ ]:
file_index_to_filename_map = {}
for key in source_file_encoding_dict.keys():
    file_index_to_filename_map[source_file_encoding_dict[key]] = os.path.basename(key)


f0_bins = get_f0_bins()
dataset_separated_histograms = {}
dataset_separated_unique_segments = {}
dataset_separated_total_segments = {}

for file_index in np.unique(file_indexes):
    f0_values_from_file_idx = f0_values[file_indexes == file_index]
    segment_indexes_from_file_idx = segment_indexes[file_indexes == file_index]
    counts, bins = np.histogram(f0_values_from_file_idx, bins=f0_bins)
    dset_key = file_index_to_filename_map[file_index]
    dataset_separated_histograms[dset_key] = counts
    dataset_separated_unique_segments[dset_key] = np.unique(segment_indexes_from_file_idx).shape[0]
    dataset_separated_total_segments[dset_key] = segment_indexes_from_file_idx.shape[0]

dataset_details = {
    'RWC': {
        'key': 'f0_segments_2019AUG16_rwc.hdf5',
        'plot_kwargs': {'color': [0, 0.8, 0]},
    },
    'NSYNTH': {
        'key': 'f0_segments_2019AUG16_nsynth.hdf5',
        'plot_kwargs': {'color': [0, 0.6, 0]}
    },
    'WSJ': {
        'key': 'f0_segments_2019AUG16_wsj.hdf5',
        'plot_kwargs': {'color': [0.6, 0.6, 0.6]}
    },
    'SWC': {
        'key': 'f0_segments_2019AUG16_swc.hdf5',
        'plot_kwargs': {'color': [0.4, 0.4, 0.4]}
    },
    'CSLUKIDS': {
        'key': 'f0_segments_2019NOV16_cslu_kids.hdf5',
        'plot_kwargs': {'color': [0.2, 0.2, 0.2]}
    },
    'CMUKIDS': {
        'key': 'f0_segments_2019NOV22_cmu_kids.hdf5',
        'plot_kwargs': {'color': [0.8, 0.8, 0.8]}
    },
}

dataset_list = ['RWC', 'NSYNTH', 'WSJ', 'SWC', 'CSLUKIDS', 'CMUKIDS']
dataset_list.reverse()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
x = np.arange(0, len(counts))
bottom = np.zeros_like(x)
for dataset in dataset_list:
    key = dataset_details[dataset]['key']
    if key in dataset_separated_histograms.keys():
        y = dataset_separated_histograms[key]
        plot_kwargs = dataset_details[dataset]['plot_kwargs']
        label = '{:s} ({} total; {} unique; {:.1f} mean repeats)'.format(
            dataset,
            dataset_separated_total_segments[key],
            dataset_separated_unique_segments[key],
            dataset_separated_total_segments[key] / dataset_separated_unique_segments[key])
        ax.fill_between(x, y1=bottom, y2=bottom+y, **plot_kwargs, lw=0, label=label)
        bottom = bottom + y
    else:
        print(key)

ax.legend(loc='upper right', framealpha=1, facecolor='w', edgecolor='w', fontsize=10)
ax.set_xlim([x[0], x[-1]])
ax.set_ylim([0, np.max(bottom)])
ax.set_ylabel('Number of stimuli')
ax.set_xlabel('F0 bin (Hz)')
class_indexes = np.linspace(x[0], x[-1], 15, dtype=int)
f0_class_labels = ['{:.0f}'.format(_) for _ in f0_bins[class_indexes]]
ax.set_xticks(class_indexes)
ax.set_xticklabels(f0_class_labels)
plt.tight_layout()
plt.show()

# save_dir = '/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_12_05_PNDv08_archSearch01/'
# save_fn = '2019NOV27_PND_v08_dataset_composition.pdf'
# print(os.path.join(save_dir, save_fn))
# fig.savefig(os.path.join(save_dir, save_fn), bbox_inches='tight')


In [ ]:
# Check how many bins are spanned by speech-only and music-only datasets
f0_bin_min=80
f0_bin_max=1e3
f0_min=80
f0_max=450.91752190019395
binwidth_in_octaves=1/192
f0_values = np.arange(f0_min, f0_max+1e-2, 1e-2)
f0_bins = get_f0_bins(f0_min=f0_bin_min, f0_max=f0_bin_max, binwidth_in_octaves=binwidth_in_octaves)
f0_labels = f0_to_label(f0_values, f0_bins, right=False)
# Slightly hacky way to determine the correct value of f0_max to ensure all bins are equally wide
f0_min_label = np.squeeze(np.argwhere(f0_bins >= f0_min))[0]
f0_max_label = np.squeeze(np.argwhere(f0_bins < f0_max))[-1] + 1
f0_min_label, f0_max_label


In [ ]:
import sys
import os
import json
import numpy as np
import glob

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

from stimuli_generate_synthetic_tones import get_bandpass_filter_frequency_response
from stimuli_generate_synthetic_tones import bernoxMovingFilter_bandpass_complex_tone
from stimuli_generate_synthetic_tones import generate_bernoxMovingFilter_dataset

f0 = 300
fs = 32000
dur = 0.150
low_harm = 30


frequency_response_in_dB = get_bandpass_filter_frequency_response(2.5e3, 3.5e3, fs=fs, order=4)
y, harms = bernoxMovingFilter_bandpass_complex_tone(f0, fs, dur, low_harm,
                                                    frequency_response_in_dB=frequency_response_in_dB,
                                                    threshold_dBSPL=33.3, component_dBSL=15.)
noise = util_stimuli.modified_uniform_masking_noise(fs, dur, dBHzSPL=15,
                                                    attenuation_start=600,
                                                    attenuation_slope=2)
y = noise + y
plt.figure()
plt.plot(y)
plt.show()


fxx, pxx = util_stimuli.power_spectrum(y, fs)
plt.figure(figsize=(15, 5))
plt.plot(fxx, pxx)
# plt.xlim([harms[0]-2, harms[-1]+2])
plt.show()

ipd.display(ipd.Audio(y, rate=fs))


In [ ]:
import numpy as np

f0_min=80.
f0_max=1e3
step_size_in_octaves=1/(12*16*64)
phase_modes=['sine', 'rand']
    
phase_mode_encoding = {'sine':0, 'rand':1, 'sch':2, 'cos':3, 'alt':4}
phase_mode_decoding = {0:'sine', 1:'rand', 2:'sch', 3:'cos', 4:'alt'}
# Determine list of f0 and phase values
f0s = np.arange(0, np.log2(f0_max / f0_min), step_size_in_octaves)
f0s = f0_min * (np.power(2, f0s))
f0_list = np.tile(f0s, len(phase_modes))
phase_list = []
for phase in phase_modes:
    phase_list.append(phase_mode_encoding[phase] * np.ones_like(f0s))
phase_list = np.concatenate(phase_list, axis=0)
N = f0_list.shape[0]
print(N, 12*16)

FDB =np.log10(0)
threshold_dBSPL=33.3
component_dBSL=15.
harmonic_dBSPL = threshold_dBSPL + component_dBSL + FDB
amplitudes = 20e-6 * np.power(10, (harmonic_dBSPL/20))
amplitudes, harmonic_dBSPL

In [ ]:
import h5py
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn = '/om/user/msaddler/data_pitchnet/bernox2005/MovingFilter_f0min100_f0max300/MovingFilter_f0min100_f0max300.hdf5'
fn = 'test.hdf5'
f = h5py.File(fn, 'r')
fsig = f['tone_in_noise']
idx = np.random.randint(2600)#fsig.shape[0])

y = fsig[idx]
fs = f['config_tone/fs'][0]

f0 = f['f0'][idx]
base_f0 = f['base_f0'][idx]
low_harm = f['low_harm'][idx]
min_harm = f['min_audible_harm'][idx]
max_harm = f['max_audible_harm'][idx]
phase_mode = f['phase_mode'][idx]

print('F0={}, LOW_HARM={}, MIN_HARM={}, MAX_HARM={}, PHASE_MODE={}'.format(f0, low_harm, min_harm, max_harm, phase_mode))

plt.figure(figsize=(10, 3))
plt.plot(y)
plt.show()

plt.figure(figsize=(10, 3))
fxx, pxx = util_stimuli.power_spectrum(y, fs)
plt.plot(fxx, pxx)

fidx = np.logical_and(fxx >= low_harm*f0, fxx <= max_harm*f0)
plt.plot(fxx[fidx], pxx[fidx], 'y')

fidx = np.logical_and(fxx >= min_harm*f0, fxx <= max_harm*f0)
plt.plot(fxx[fidx], pxx[fidx], 'r')

plt.show()

f.close()

ipd.display(ipd.Audio(y, rate=fs))

In [ ]:
import sys
import os
import json
import numpy as np
import glob

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

from stimuli_generate_synthetic_tones import get_bandpass_filter_frequency_response
from stimuli_generate_synthetic_tones import shift_bandpass_filter_frequency_response

fs=32e3
dur=0.150

threshold_dBSPL=33.3
component_dBSL=15.
noise_dBHzSPL=15.
noise_attenuation_start=600.
noise_attenuation_slope=2

# Build the unmodified Bernstein & Oxenham (2005) filter frequency response function
highpass_filter_cutoff=2.5e3*2
lowpass_filter_cutoff=3.5e3*2
filter_order=4
frequency_response_in_dB = get_bandpass_filter_frequency_response(highpass_filter_cutoff,
                                                                  lowpass_filter_cutoff,
                                                                  fs=fs,
                                                                  order=filter_order)

# Define lists of f0 and lowest harmonic number values of interest
f0_min=100.
f0_max=300.
step_size_in_octaves=2/12
phase_modes=['sine', 'rand']
low_harm_min=1
low_harm_max=30
base_low_harm_numbers = np.arange(low_harm_min, low_harm_max + 1)
base_f0s = np.arange(0, np.log2(f0_max / f0_min), step_size_in_octaves)
base_f0s = f0_min * (np.power(2, base_f0s))
print(len(base_f0s) * len(base_low_harm_numbers) * len(phase_modes))

# For each pair of base f0 and base low_harm, shift the filter
def get_valid_f0_range(frequency_response_in_dB, minimum_gain, low_harm, fs=32e3):
    freq_vector = np.arange(0, fs/2)
    gain_vector = frequency_response_in_dB(freq_vector)
    passed_freqs = freq_vector[gain_vector > minimum_gain]
    fl = np.min(passed_freqs)
    fh = np.max(passed_freqs)
    
    min_f0 = fl / low_harm
    if low_harm > 1: max_f0 = fl / (low_harm - 1)
    else: max_f0 = fh / low_harm
    return [min_f0, max_f0]


def compute_low_harm(f0, frequency_response_in_dB, minimum_gain, fs=32e3):
    harm_freqs = np.arange(f0, fs/2, f0)
    harm_nums = np.array(harm_freqs / f0).astype(int)
    harm_gain = frequency_response_in_dB(harm_freqs)
    return np.min(harm_nums[harm_gain > minimum_gain])
    

f0_test_range = np.array([0.925, 1.075])
filter_counter = 0
for base_lh in base_low_harm_numbers:
    for base_f0 in base_f0s:
        filter_counter += 1
        desired_fl = base_lh * base_f0
        desired_fl_gain_in_dB = -1 * component_dBSL
#         print(filter_counter, base_lh, base_f0, desired_fl)
        
        new_freq_resp = shift_bandpass_filter_frequency_response(desired_fl, desired_fl_gain_in_dB,
                                                                 fs=fs, unshifted_passband=None,
                                                                 frequency_response_in_dB=frequency_response_in_dB)
        lhtmp_range = []
        f0tmp_range = []
        for f0tmp in base_f0*f0_test_range:
            lhtmp = compute_low_harm(f0tmp, new_freq_resp, desired_fl_gain_in_dB, fs=fs)
            lhtmp_range.append(lhtmp)
            f0tmp_range.append(f0tmp)
        
        print(base_lh, int(base_f0), f0tmp_range, lhtmp_range)
        
#         f0_range = get_valid_f0_range(new_freq_resp, desired_fl_gain_in_dB, base_lh, fs=fs)
        
#         f0_range = 100*np.array(f0_range)/base_f0
        
#         print(base_lh, int(base_f0), f0_range.astype(int))
        


plt.figure()
fr = np.arange(0, fs/2)
plt.plot(fr, frequency_response_in_dB(fr))
plt.ylim([-15, 0])
plt.show()

In [ ]:
def compute_low_harm(f0, frequency_response_in_dB, minimum_gain, fs=32e3):
    harm_freqs = np.arange(f0, fs/2, f0)
    harm_nums = np.array(harm_freqs / f0).astype(int)
    harm_gain = frequency_response_in_dB(harm_freqs)
    return np.min(harm_nums[harm_gain > minimum_gain])
    
compute_low_harm(600, frequency_response_in_dB, -15, fs=fs)

In [ ]:
base_f0_min=100.
base_f0_max=300.

np.power(2, np.linspace(np.log2(base_f0_min), np.log2(base_f0_max), 20))

delta_f0_range = [0.925, 1.075]
delta_f0_n = 101

z = 150*np.linspace(delta_f0_range[0], delta_f0_range[1], delta_f0_n)

percent_diffs = []
for tmp1 in z:
    for tmp2 in z:
        percent_diffs.append(100*np.abs(tmp1 - tmp2)/tmp2)

plt.figure()
plt.hist(percent_diffs)
plt.show()

In [ ]:
def generate_BernsteinOxenhamFixedFilter_dataset(hdf5_filename, fs=32e3, dur=0.150, phase_modes=['sine', 'rand'],
                                                 low_harm_min=1, low_harm_max=30,
                                                 base_f0_min=100.0, base_f0_max=300.0, base_f0_n=12,
                                                 delta_f0_min=0.925, delta_f0_max=1.075, delta_f0_n=101,
                                                 highpass_filter_cutoff=2.5e3, lowpass_filter_cutoff=3.5e3,
                                                 filter_order=4, threshold_dBSPL=33.3, component_dBSL=15.,
                                                 noise_dBHzSPL=15., noise_attenuation_start=600.,
                                                 noise_attenuation_slope=2, disp_step=100):
    '''
    '''
    # Define encoding / decoding dictionaries for phase_mode
    phase_mode_encoding = {'sine':0, 'rand':1, 'sch':2, 'cos':3, 'alt':4}
    phase_mode_decoding = {0:'sine', 1:'rand', 2:'sch', 3:'cos', 4:'alt'}
    # Define lists of unique phase modes and low harm numbers
    unique_ph_list = np.array([phase_mode_encoding[p] for p in phase_modes])
    unique_lh_list = np.arange(low_harm_min, low_harm_max + 1)
    # Define list of "base f0" values (Hz), which are used to set the filters
    base_f0_list = np.power(2, np.linspace(np.log2(base_f0_min), np.log2(base_f0_max), base_f0_n))
    # Define list of "delta f0" values (fraction of f0)
    delta_f0_list = np.linspace(delta_f0_min, delta_f0_max, delta_f0_n)
    # Compute number of stimuli (all combinations of low_harm, base_f0, delta_f0, and phase_mode)
    N = len(unique_ph_list) * len(unique_lh_list) * len(base_f0_list) * len(delta_f0_list)
    # Calculate the "unshifted" bandpass filter frequency response function to use as a baseline
    baseline_freq_response = get_bandpass_filter_frequency_response(highpass_filter_cutoff,
                                                                    lowpass_filter_cutoff,
                                                                    fs=fs, order=filter_order)
    # Prepare data_dict and config_key_pair_list for hdf5 filewriting
    data_dict = {
        'config_tone/fs': fs,
        'config_tone/dur': dur,
        'config_tone/low_harm_min': low_harm_min,
        'config_tone/low_harm_max': low_harm_max,
        'config_tone/base_f0_min': base_f0_min,
        'config_tone/base_f0_max': base_f0_max,
        'config_tone/base_f0_n': base_f0_n,
        'config_tone/delta_f0_min': delta_f0_min,
        'config_tone/delta_f0_max': delta_f0_max,
        'config_tone/delta_f0_n': delta_f0_n,
        'config_tone/unshifted_highpass_filter_cutoff': highpass_filter_cutoff,
        'config_tone/unshifted_lowpass_filter_cutoff': lowpass_filter_cutoff,
        'config_tone/unshifted_filter_order': filter_order,
        'config_tone/threshold_dBSPL': threshold_dBSPL,
        'config_tone/component_dBSL': component_dBSL,
        'config_tone/noise_dBHzSPL': noise_dBHzSPL,
        'config_tone/noise_attenuation_start': noise_attenuation_start,
        'config_tone/noise_attenuation_slope': noise_attenuation_slope,
    }
    config_key_pair_list = [(k, k) for k in data_dict.keys()]
    data_key_pair_list = [] # Will be populated right before initializing hdf5 file
    # Main loop to generate the bandpass filtered tones
    itrN = 0
    for lh in unique_lh_list:
        for base_f0 in base_f0_list:
            # Compute fixed filter's frequency response using low harm number and base_f0
            desired_fl = base_f0 * low_harm
            desired_fl_gain_in_dB = -1 * component_dBSL
            fixed_freq_response = shift_bandpass_filter_frequency_response(
                desired_fl, desired_fl_gain_in_dB, fs=fs, unshifted_passband=None,
                frequency_response_in_dB=baseline_freq_response)
            for delta_f0 in delta_f0_list:
                for ph in unique_ph_list:
                    # Construct signal with specified f0 and phase mode
                    f0 = base_f0 * delta_f0
                    signal, audible_harmonic_numbers = bernox2005_bandpass_complex_tone(
                        f0, fs, dur, frequency_response_in_dB=fixed_freq_response,
                        threshold_dBSPL=threshold_dBSPL, component_dBSL=component_dBSL,
                        phase_mode=phase_mode_decoding[ph])
                    # Construct modified uniform masking noise
                    noise = util_stimuli.modified_uniform_masking_noise(
                        fs, dur, dBHzSPL=noise_dBHzSPL,
                        attenuation_start=noise_attenuation_start,
                        attenuation_slope=noise_attenuation_slope)
                    # Add signal + noise and metadata to data_dict for hdf5 filewriting
                    tone_in_noise = signal + noise
                    data_dict['tone_in_noise'] = tone_in_noise.astype(np.float32)
                    data_dict['f0'] = f0
                    data_dict['base_f0'] = base_f0
                    data_dict['delta_f0'] = delta_f0
                    data_dict['phase_mode'] = int(ph)
                    data_dict['low_harm'] = int(lh)
                    data_dict['min_audible_harm'] = int(np.min(audible_harmonic_numbers))
                    data_dict['max_audible_harm'] = int(np.max(audible_harmonic_numbers))
                    # Initialize the hdf5 file on the first iteration
                    if itrN == 0:
                        print('[INITIALIZING]: {}'.format(hdf5_filename))
                        for k in data_dict.keys():
                            if not (k, k) in config_key_pair_list:
                                data_key_pair_list.append((k, k))
                        initialize_hdf5_file(hdf5_filename, N, data_dict, file_mode='w',
                                             data_key_pair_list=data_key_pair_list,
                                             config_key_pair_list=config_key_pair_list)
                        hdf5_f = h5py.File(hdf5_filename, 'r+')
                    # Write each data_dict to hdf5 file
                    write_example_to_hdf5(hdf5_f, data_dict, itrN,
                                          data_key_pair_list=data_key_pair_list)
                    if itrN % disp_step == 0:
                        print('... signal {} of {}'.format(itrN, N))
                    itrN += 1
    # Close hdf5 file
    hdf5_f.close()
    print('[END]: {}'.format(hdf5_filename))

generate_BernsteinOxenhamFixedFilter_dataset('')

In [ ]:

base_f0_min=100.0
base_f0_max=300.0
base_f0_n=10

delta_f0_min=0.94
delta_f0_max=1.06
delta_f0_n=121

base_f0_list = np.power(2, np.linspace(np.log2(base_f0_min), np.log2(base_f0_max), base_f0_n))
delta_f0_list = np.linspace(delta_f0_min, delta_f0_max, delta_f0_n)

f0_list = []
for bf0 in base_f0_list:
    for df0 in delta_f0_list:
        f0_list.append(bf0*df0)
plt.figure(figsize=(15, 10))
plt.plot(f0_list, '.')
plt.show()

delta_f0_list
# base_f0_list

In [ ]:
import numpy as np
# Define stimulus-specific parameters
list_f_carrier = [0.0, 4000.0, 6350.0, 10080.0]
f0_min=80.
f0_max=320.
step_size_in_octaves=1/(12*16*16)
f0s = np.arange(0, np.log2(f0_max / f0_min), step_size_in_octaves)
f0s = f0_min * (np.power(2, f0s))
list_f_envelope = list(f0s)

# Iterate over all combinations of stimulus-specific parameters
N = len(list_f_carrier) * len(list_f_envelope)
N

In [ ]:
import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn = 'test.hdf5'
f = h5py.File(fn, 'r')
fsig = f['tone_in_noise']

for _ in range(20):
    idx = np.random.randint(fsig.shape[0])

    y = fsig[idx]
    fs = f['config_tone/fs'][0]

    f0 = f['f0'][idx]
    filter_fl = f['filter_fl'][idx]
    phase_mode = f['phase_mode'][idx]

#     print('F0={}, filter_fl={}, PHASE_MODE={}'.format(f0, filter_fl, phase_mode))

#     plt.figure(figsize=(10, 3))
#     plt.plot(y)
#     plt.show()

    plt.figure(figsize=(10, 3))
    fxx, pxx = util_stimuli.power_spectrum(y, fs)
    plt.plot(fxx, pxx)

    plt.show()

f.close()

ipd.display(ipd.Audio(y, rate=fs))

print(20*np.log10(util_stimuli.rms(y)/20e-6))

In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn_dir = '/om/user/msaddler/data_pitchnet/'

fn = os.path.join(fn_dir, 'bernox2005/FixedFilter_f0min100_f0max300/FixedFilter_f0min100_f0max300.hdf5')
f = h5py.File(fn, 'r')

signal_key = 'tone_in_noise'
sr = 32000
vals_f0 = f['f0'][:]
vals_lharm = f['low_harm'][:]
vals_phase = f['phase_mode'][:]

IDX = np.logical_and(vals_phase == 0, vals_lharm == 5)
IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200, vals_f0<200.2))
IDX = np.squeeze(np.argwhere(IDX))
print(IDX)

fontsize=18
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 1.4))
cx = 0.6
COLOR_LIST = [[cx, cx, cx], [0, 0, 0]]

IDX = 11622#47920#64860
for cidx, IDX in enumerate([11620]):#, 64860]):
    y = f[signal_key][IDX]
    f0 = vals_f0[IDX]
    fxx, pxx  = util_stimuli.power_spectrum(y, sr)

    plot_kwargs = {'lw':4, 'color':COLOR_LIST[cidx]}
    ax.plot(fxx/f0, pxx, **plot_kwargs)
    
    print('F0={}, low_harm={}'.format(vals_f0[IDX], vals_lharm[IDX]))

ax.set_xlim([0, 50.5])
ax.set_ylim([16, 44])
ax.set_xticks(np.arange(0, 50, 5))
# ax.set_yticks(np.arange(0, 45, 15))
ax.tick_params(axis='both', labelsize=fontsize-2)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.set_xlabel('Harmonic number (frequency / F0)', fontsize=fontsize)
ax.set_ylabel('dB/Hz SPL', fontsize=fontsize)
plt.show()

# fig.savefig('../assets_psychophysics/figures/bernox_spectrum_pitch250_lharm05.pdf', bbox_inches='tight')


IDX = 47920+12
y = f[signal_key][IDX]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7.5, 1.5))
plot_kwargs = {'lw':2, 'color':'k'}
n = 400
offset=110
ax.plot(y[offset:offset+n], **plot_kwargs)
ax.set_xlim([0, n])
ax.set_xticks([])
ax.set_yticks([])
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
plt.show()

# fig.savefig('../assets_psychophysics/figures/bernox_waveform_lharm20_pitch250_sine.pdf', bbox_inches='tight')

f.close()


In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn_dir = '/om/user/msaddler/data_pitchnet/'

fn = os.path.join(fn_dir, 'mooremoore2003/MooreMoore2003_frequencyShiftedComplexes_f0_080to480Hz/MooreMoore2003_frequencyShiftedComplexes_f0_080to480Hz.hdf5')
f = h5py.File(fn, 'r')

signal_key = 'stimuli/signal'
sr = 32000
vals_f0 = f['f0'][:]
vals_harm_cent = f['spectral_envelope_centered_harmonic'][:]
vals_f0_shift = f['f0_shift'][:]

IDX = np.logical_and(np.logical_or(vals_f0_shift == 0.0, vals_f0_shift == 0.24), vals_harm_cent == 11)
IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200, vals_f0<200.1))
IDX = np.squeeze(np.argwhere(IDX))
print(IDX)


fontsize=18
cx = 0.80
plot_dict = {
    'preshift_RES': {'IDX_LIST': [14217], 'COLOR_LIST': [[1, 0, 0]]},
    'preshift_INT': {'IDX_LIST': [42014], 'COLOR_LIST': [[0, 0, 1]]},
    'preshift_UNRES': {'IDX_LIST': [69811], 'COLOR_LIST': [[0, 0, 0]]},
    
    'postshift_RES': {'IDX_LIST': [14217, 14223], 'COLOR_LIST': [[1, cx, cx], [1, 0, 0]]},
    'postshift_INT': {'IDX_LIST': [42014, 42020], 'COLOR_LIST': [[cx, cx, 1], [0, 0, 1]]},
    'postshift_UNRES': {'IDX_LIST': [69811, 69817], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]}
}

save_fn_pattern = '../assets_psychophysics/figures/freqshift_spectrum_{}.pdf'

for key in plot_dict.keys():    
    IDX_LIST = plot_dict[key]['IDX_LIST']
    COLOR_LIST = plot_dict[key]['COLOR_LIST']

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 1.25))
    for cidx, IDX in enumerate(IDX_LIST):
        y = f[signal_key][IDX]
        f0 = vals_f0[IDX]
        fxx, pxx  = util_stimuli.power_spectrum(y, sr)

        plot_kwargs = {'lw':3, 'color':COLOR_LIST[cidx]}
        ax.plot(fxx/f0, pxx, **plot_kwargs)

    ax.set_xlim([0, 20])
    ax.set_ylim([25, 55])
    ax.set_xticks(np.arange(0, 21, 2))
    # ax.set_yticks(np.arange(0, 45, 15))
    ax.tick_params(axis='both', labelsize=fontsize-2)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel('Harmonic number (frequency / F0)', fontsize=fontsize)
    ax.set_ylabel('dB/Hz SPL', fontsize=fontsize)

    plt.show()
    print(save_fn_pattern.format(key))
#     fig.savefig(save_fn_pattern.format(key), bbox_inches='tight')

f.close()


In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn_dir = '/om/user/msaddler/data_pitchnet/'

fn = os.path.join(fn_dir, 'moore1985/Moore1985_MistunedHarmonics_v01/Moore1985_MistunedHarmonics_v01.hdf5')
f = h5py.File(fn, 'r')

for v in f.values():
    print(v)

signal_key = 'stimuli/signal'
sr = 32000
vals_f0 = f['f0'][:]
vals_mistuned_harm = f['mistuned_harm'][:]
vals_mistuned_pct = f['mistuned_pct'][:]

IDX = np.logical_and(np.logical_or(vals_mistuned_pct == 0.0, vals_mistuned_pct == 3.0), vals_mistuned_harm == 12)
IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200.0, vals_f0<=200.1))
IDX = np.squeeze(np.argwhere(IDX))
print(IDX)



fontsize=18
cx = 0.80
plot_dict = {
    'tmp1': {'IDX_LIST': [10415], 'COLOR_LIST': [[0, 0, 0]]},
    'tmp2': {'IDX_LIST': [10415, 12017], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]},
    'tmp3': {'IDX_LIST': [10415, 39785], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]},
    'tmp4': {'IDX_LIST': [10415, 81437], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]},
}


for key in sorted(plot_dict.keys()):    
    IDX_LIST = plot_dict[key]['IDX_LIST']
    COLOR_LIST = plot_dict[key]['COLOR_LIST']

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 1.5))
    for cidx, IDX in enumerate(IDX_LIST):
        y = f[signal_key][IDX]
        f0 = vals_f0[IDX]
        fxx, pxx  = util_stimuli.power_spectrum(y, sr)

        plot_kwargs = {'lw':3, 'color':COLOR_LIST[cidx]}
        ax.plot(fxx/f0, pxx, **plot_kwargs)

    ax.set_xlim([0, 13])
    ax.set_ylim([25, 55])
    ax.set_xticks(np.arange(0, 13, 2))
    # ax.set_yticks(np.arange(0, 45, 15))
    ax.tick_params(axis='both', labelsize=fontsize-2)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel('Harmonic number (frequency / F0)', fontsize=fontsize)
    ax.set_ylabel('dB/Hz SPL', fontsize=fontsize)

    plt.show()
#     save_fn_pattern = '../assets_psychophysics/figures/mistunedharmonics_spectrum_{}.pdf'
#     print(save_fn_pattern.format(key))
#     fig.savefig(save_fn_pattern.format(key), bbox_inches='tight')

f.close()


In [ ]:
import sys
sys.path.append('/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/pitchnetDataset')
from dataset_util import get_f0_bins, f0_to_label, f0_to_octave, f0_to_normalized
from dataset_util import label_to_f0, octave_to_f0, normalized_to_f0

In [ ]:
x = get_f0_bins()

(x[1]-x[0]) / x[1]

# x[0], x[1]

In [ ]:
1/0.062499999999998744

In [ ]:
import os
import sys
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import glob
import h5py
import IPython.display as ipd

from stimuli_generate_BernsteinOxenhamFixedFilter import *

fs=32000
dur = 0.5

phase_mode = 'sine'

highpass_filter_cutoff=2.5e3
lowpass_filter_cutoff=3.5e3
filter_order=4
threshold_dBSPL=33.3
component_dBSL=15.0
noise_dBHzSPL=15.0
noise_attenuation_start=600.0
noise_attenuation_slope=2



diff = 1.03
(f0_1, f0_2) = (150, 150*diff)
base_f0 = 150
lh = 28
nrep = 1
padlen = int(fs/10)
pin = np.array([])

baseline_freq_response = get_bandpass_filter_frequency_response(highpass_filter_cutoff,
                                                                lowpass_filter_cutoff,
                                                                fs=fs, order=filter_order)
desired_fl = base_f0 * lh
desired_fl_gain_in_dB = -1 * component_dBSL
fixed_freq_response = shift_bandpass_filter_frequency_response(
    desired_fl, desired_fl_gain_in_dB, fs=fs, unshifted_passband=None,
    frequency_response_in_dB=baseline_freq_response)

for _ in range(nrep):
    signal_1, audible_harmonic_numbers = bernox2005_bandpass_complex_tone(
        f0_1, fs, dur, frequency_response_in_dB=fixed_freq_response,
        threshold_dBSPL=threshold_dBSPL, component_dBSL=component_dBSL,
        phase_mode=phase_mode)
    signal_1 = np.pad(signal_1, (padlen, padlen), 'constant', constant_values=(0, 0))
    
    signal_2, audible_harmonic_numbers = bernox2005_bandpass_complex_tone(
        f0_2, fs, dur, frequency_response_in_dB=fixed_freq_response,
        threshold_dBSPL=threshold_dBSPL, component_dBSL=component_dBSL,
        phase_mode=phase_mode)
    signal_2 = np.pad(signal_2, (padlen, padlen), 'constant', constant_values=(0, 0))
    
    pin = np.append(pin, signal_1, axis=0)
    pin = np.append(pin, signal_2, axis=0)
# # Construct modified uniform masking noise
# noise = util_stimuli.modified_uniform_masking_noise(
#     fs, dur, dBHzSPL=noise_dBHzSPL,
#     attenuation_start=noise_attenuation_start,
#     attenuation_slope=noise_attenuation_slope)

print(lh, f0_1, f0_2, base_f0)
ipd.display(ipd.Audio(pin, rate=fs))

# fn_out = '/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_10_01_cog_lunch_figures/bernox_demo_lharm28_sine_150Hz_3pct.wav'
# print(fn_out)
# import scipy.io.wavfile
# scipy.io.wavfile.write(fn_out, fs, np.squeeze(pin))

In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
sys.path.append('/code_location/multi_gpu')
import functions_erbfilter as erb


sr = 32000
n = 40
signal_length = 4800
min_cf = 125
max_cf = 14000
bandwidth_scale_factor = 1.0
filts, cfs, freqs = erb.make_roex_filters(signal_length, sr, n,  min_cf=min_cf, max_cf=max_cf,
                                          bandwidth_scale_factor=bandwidth_scale_factor,
                                          padding_size=None, full_filter=False, dc_ramp_cutoff=30)
filts = 20*np.log10(filts)



ylimits = [18, 42]
xlimits = [0, 11e3]
filts = filts + ylimits[-1] - 1

fn_dir = '/om/user/msaddler/data_pitchnet/'

fn = os.path.join(fn_dir, 'bernox2005/FixedFilter_f0min100_f0max300/FixedFilter_f0min100_f0max300.hdf5')
f = h5py.File(fn, 'r')

signal_key = 'tone_in_noise'
sr = 32000
vals_f0 = f['f0'][:]
vals_lharm = f['low_harm'][:]
vals_phase = f['phase_mode'][:]

IDX_list = []
for low_harm in [4, 27]:
    IDX = np.logical_and(vals_phase == 0, vals_lharm == low_harm)
    IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200, vals_f0<200.2))
    IDX = np.squeeze(np.argwhere(IDX))
    print('low_harm={}, IDX={}'.format(low_harm, IDX))
    IDX_list.append(int(IDX))

fontsize=16
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 1.25))
cx = 0.6
COLOR_LIST = [[cx, cx, cx], [0, 0, 0]]

FILTERBANK_COLOR = [0.85, 0.9, 0.85]

for fidx in range(n):
    ax.plot(freqs, filts[:, fidx], color=FILTERBANK_COLOR, lw=2)

for cidx, IDX in enumerate(IDX_list):
    y = f[signal_key][IDX]
    f0 = vals_f0[IDX]
    fxx, pxx  = util_stimuli.power_spectrum(y, sr)

    plot_kwargs = {'lw':3, 'color':COLOR_LIST[cidx]}
    ax.plot(fxx, pxx, **plot_kwargs)
    
    print('F0={}, low_harm={}'.format(vals_f0[IDX], vals_lharm[IDX]))

ax.set_xlim(xlimits)
ax.set_ylim(ylimits)
ax.set_xticks(np.arange(xlimits[0], xlimits[1]+1, 2000), minor=False)
ax.set_xticks(np.arange(xlimits[0], xlimits[1]+1, 200), minor=True)
ax.set_yticks([20, 40])
ax.tick_params(axis='both', labelsize=fontsize-2, which='major', length=6)
ax.tick_params(axis='both', labelsize=fontsize-2, which='minor', length=3)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.set_xlabel('Frequency (Hz)', fontsize=fontsize)
ax.set_ylabel('dB/Hz SPL', fontsize=fontsize)
plt.show()

# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_10_17_APAN_poster/schematic_bernox_spectra_with_filterbank.pdf', bbox_inches='tight')



IDX_list = []
for phase_mode in [1, 0]:
    IDX = np.logical_and(vals_phase == phase_mode, vals_lharm == 20)
    IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200, vals_f0<200.2))
    IDX = np.squeeze(np.argwhere(IDX))
    print('low_harm={}, IDX={}'.format(low_harm, IDX))
    IDX_list.append(int(IDX))

fig, ax = plt.subplots(nrows=len(IDX_list), ncols=1, figsize=(4, 2.5),
                       sharex=True, sharey=True)


n = 380
offset=155
for ax_idx, IDX in enumerate(IDX_list):
    y = f[signal_key][IDX]

    plot_kwargs = {'lw':2, 'color':'k'}
    ax[ax_idx].plot(y[offset:offset+n], **plot_kwargs)
    ax[ax_idx].set_xlim([0, n])
    ax[ax_idx].set_xticks([])
    ax[ax_idx].set_yticks([])
    ax[ax_idx].spines["top"].set_visible(False)
    ax[ax_idx].spines["right"].set_visible(False)
    ax[ax_idx].spines["bottom"].set_visible(False)
    ax[ax_idx].spines["left"].set_visible(False)
    
plt.tight_layout()
plt.show()

# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_10_17_APAN_poster/schematic_bernox_waveform_sine_rand_lowharm20.pdf', bbox_inches='tight')

f.close()


In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn_dir = '/om/user/msaddler/data_pitchnet/'

fn = os.path.join(fn_dir, 'mooremoore2003/MooreMoore2003_frequencyShiftedComplexes_f0_080to480Hz/MooreMoore2003_frequencyShiftedComplexes_f0_080to480Hz.hdf5')
f = h5py.File(fn, 'r')

signal_key = 'stimuli/signal'
sr = 32000
vals_f0 = f['f0'][:]
vals_harm_cent = f['spectral_envelope_centered_harmonic'][:]
vals_f0_shift = f['f0_shift'][:]

# IDX = np.logical_and(np.logical_or(vals_f0_shift == 0.0, vals_f0_shift == 0.24), vals_harm_cent == 11)
# IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200, vals_f0<200.1))
# IDX = np.squeeze(np.argwhere(IDX))
# print(IDX)


fontsize=20
cx = 0.75
plot_dict = {
#     'preshift_RES': {'IDX_LIST': [14217], 'COLOR_LIST': [[1, 0, 0]]},
#     'preshift_UNRES': {'IDX_LIST': [69811], 'COLOR_LIST': [[0, 0, 0]]},
    
    'postshift_RES': {'IDX_LIST': [14217, 14223], 'COLOR_LIST': [[1, cx, cx], [1, 0, 0]]},
    'postshift_UNRES': {'IDX_LIST': [69811, 69817], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]}
}


sr = 32000
n = 40
signal_length = 4800
min_cf = 125
max_cf = 14000
bandwidth_scale_factor = 1.0
filts, cfs, freqs = erb.make_roex_filters(signal_length, sr, n,  min_cf=min_cf, max_cf=max_cf,
                                          bandwidth_scale_factor=bandwidth_scale_factor,
                                          padding_size=None, full_filter=False, dc_ramp_cutoff=30)
filts = 20*np.log10(filts)

xlimits = [0, 4000]
ylimits = [25, 60]
filts = filts + ylimits[-1] - 1

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 1.2))

FILTERBANK_COLOR = [0.85, 0.9, 0.85]

for fidx in range(n):
    ax.plot(freqs, filts[:, fidx], color=FILTERBANK_COLOR, lw=2)

for key in plot_dict.keys():    
    IDX_LIST = plot_dict[key]['IDX_LIST']
    COLOR_LIST = plot_dict[key]['COLOR_LIST']

    for cidx, IDX in enumerate(IDX_LIST):
        y = f[signal_key][IDX]
        f0 = vals_f0[IDX]
        fxx, pxx  = util_stimuli.power_spectrum(y, sr)

        plot_kwargs = {'lw':3, 'color':COLOR_LIST[cidx]}
        ax.plot(fxx, pxx, **plot_kwargs)

ax.set_xlim(xlimits)
ax.set_ylim(ylimits)
ax.set_xticks(np.arange(xlimits[0], xlimits[1]+1, 1000), minor=False)
ax.set_xticks(np.arange(xlimits[0], xlimits[1]+1, 200), minor=True)

ax.set_yticks([30, 55])
ax.tick_params(axis='both', labelsize=fontsize-2, which='major', length=6)
ax.tick_params(axis='both', labelsize=fontsize-2, which='minor', length=3)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.set_xlabel('Frequency (Hz)', fontsize=fontsize)
ax.set_ylabel('dB/Hz SPL', fontsize=fontsize)
plt.show()

f.close()

# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_10_17_APAN_poster/schematic_freqshift_spectra_with_filterbank.pdf', bbox_inches='tight')


In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

fn_dir = '/om/user/msaddler/data_pitchnet/'

fn = os.path.join(fn_dir, 'moore1985/Moore1985_MistunedHarmonics_v00/Moore1985_MistunedHarmonics_v00.hdf5')
f = h5py.File(fn, 'r')

signal_key = 'stimuli/signal'
sr = 32000
vals_f0 = f['f0'][:]
vals_mistuned_harm = f['mistuned_harm'][:]
vals_mistuned_pct = f['mistuned_pct'][:]

IDX = np.logical_and(np.logical_or(vals_mistuned_pct == 0.0, vals_mistuned_pct == 6.0), vals_mistuned_harm == 4)
IDX = np.logical_and(IDX, np.logical_and(vals_f0>=200, vals_f0<200.1))
IDX = np.squeeze(np.argwhere(IDX))
print(IDX)


fontsize=20
cx = 0.75
plot_dict = {
#     'preshift_200Hz_mistuned0pct': {'IDX_LIST': [31775], 'COLOR_LIST': [[0, 0, 0]]},
#     'postshift_200Hz_mistuned4pct_harm2': {'IDX_LIST': [31775, 15755], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]},
    'postshift_200Hz_mistuned6pct_harm4': {'IDX_LIST': [31775, 34979], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]},
#     'postshift_200Hz_mistuned4pct_harm6': {'IDX_LIST': [31775, 52067], 'COLOR_LIST': [[cx, cx, cx], [0, 0, 0]]},
}


sr = 32000
n = 40
signal_length = 4800
min_cf = 125
max_cf = 14000
bandwidth_scale_factor = 1.0
filts, cfs, freqs = erb.make_roex_filters(signal_length, sr, n,  min_cf=min_cf, max_cf=max_cf,
                                          bandwidth_scale_factor=bandwidth_scale_factor,
                                          padding_size=None, full_filter=False, dc_ramp_cutoff=30)
filts = 20*np.log10(filts)

xlimits = [0, 2600]
ylimits = [25, 55]
filts = filts + ylimits[-1] - 1

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 1.2))

FILTERBANK_COLOR = [0.85, 0.9, 0.85]

for fidx in range(n):
    ax.plot(freqs, filts[:, fidx], color=FILTERBANK_COLOR, lw=2)

for key in plot_dict.keys():    
    IDX_LIST = plot_dict[key]['IDX_LIST']
    COLOR_LIST = plot_dict[key]['COLOR_LIST']

    for cidx, IDX in enumerate(IDX_LIST):
        y = f[signal_key][IDX]
        f0 = vals_f0[IDX]
        fxx, pxx  = util_stimuli.power_spectrum(y, sr)

        plot_kwargs = {'lw':3, 'color':COLOR_LIST[cidx]}
        ax.plot(fxx, pxx, **plot_kwargs)

ax.set_xlim(xlimits)
ax.set_ylim(ylimits)
ax.set_xticks(np.arange(xlimits[0], xlimits[1]+1, 1000), minor=False)
ax.set_xticks(np.arange(xlimits[0], xlimits[1]+1, 200), minor=True)

ax.set_yticks([30, 50])
ax.tick_params(axis='both', labelsize=fontsize-2, which='major', length=6)
ax.tick_params(axis='both', labelsize=fontsize-2, which='minor', length=3)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.set_xlabel('Frequency (Hz)', fontsize=fontsize)
ax.set_ylabel('dB/Hz SPL', fontsize=fontsize)
plt.show()

f.close()

# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_10_17_APAN_poster/schematic_mistunedharm_spectra_with_filterbank_lowharm04_pct06.pdf', bbox_inches='tight')



In [ ]:
import sys
import os

import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

t0 = 0.0159
dur = 0.021
sr = 32000
t = np.arange(t0, t0+dur, 1/sr)

stim_list = [
    (200, 'sine'),
    (200, 'alt'),
    (400, 'sine'),
]
y_list = []
for idx, (f0, phase_mode) in enumerate(stim_list):
    print(f0, phase_mode)
    signal = np.zeros_like(t)
    freq_list = f0*np.arange(1, 16)
    if phase_mode == 'alt':
        phase_list = np.pi/2 * np.ones(len(freq_list))
        phase_list[::2] = 0
    else:
        phase_list = np.zeros(len(freq_list))

    for freq, phase in zip(freq_list, phase_list):
        if freq < sr/2:
            signal = signal + np.sin(2*np.pi*freq*t + phase)

    y_list.append(signal)

fig, ax = plt.subplots(nrows=len(y_list), ncols=1, figsize=(4, 4),
                       sharex=True, sharey=True)
for ax_idx in range(len(y_list)):
    y = y_list[ax_idx]

    plot_kwargs = {'lw':2, 'color':'k'}
    ax[ax_idx].plot(y, **plot_kwargs)
    ax[ax_idx].set_xlim([0, len(y)])
    ax[ax_idx].set_xticks([])
    ax[ax_idx].set_yticks([])
    ax[ax_idx].spines["top"].set_visible(False)
    ax[ax_idx].spines["right"].set_visible(False)
    ax[ax_idx].spines["bottom"].set_visible(False)
    ax[ax_idx].spines["left"].set_visible(False)
    
plt.tight_layout()
plt.show()

# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_10_17_APAN_poster/schematic_altphase_waveforms_200HzF0_harm1to15.pdf', bbox_inches='tight')


In [ ]:
import sys
import os
import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

import scipy.signal

t0 = 0.0
dur = 1.000
sr = 32000
t = np.arange(t0, t0+dur, 1/sr)
y = np.random.randn(t.shape[0])
print(y.shape)

fyy, pyy = util_stimuli.power_spectrum(y, sr)

plt.figure()
plt.plot(fyy, pyy)
plt.xlim([0, 8000])
plt.show()

meanrates_fs = 16000
pin_fs = 32000
y = scipy.signal.resample_poly(y, int(meanrates_fs), int(pin_fs))
y = scipy.signal.resample_poly(y, int(pin_fs), int(meanrates_fs))

fyy, pyy = util_stimuli.power_spectrum(y, sr)

plt.figure()
plt.plot(fyy, pyy)
plt.xlim([0, 8000])
plt.show()


In [ ]:
import sys
import os
import json
import numpy as np
import glob

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli


f0 = 200
fs = 32000
dur = 0.05
harmonic_numbers = np.arange(1, 21)

np.random.seed(0)
y = util_stimuli.complex_tone(f0, fs, dur, harmonic_numbers=harmonic_numbers,
                              frequencies=None, amplitudes=None, phase_mode='sine',
                              offset_start=True, strict_nyquist=True)
t = np.arange(0, y.shape[0])/fs

y = util_stimuli.set_dBSPL(y, 60)
noise = util_stimuli.modified_uniform_masking_noise(fs, dur)

y = util_stimuli.combine_signal_and_noise(y, noise, 15)

ipd.display(ipd.Audio(y, rate=fs))

ysmall = y#[0:800]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 0.75))
plot_kwargs = {'lw':1, 'color':'k'}
ax.plot(ysmall, **plot_kwargs)
ax.set_xlim([0, len(ysmall)])
ax.set_xticks([])
ax.set_yticks([])
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)

plt.tight_layout()
plt.show()
# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/tmp1.pdf', bbox_inches='tight')


fyy, pyy = stimuli_util.power_spectrum(y, fs)


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 2.5))
fontsize_labels = 16
fontsize_ticks = 12
plot_kwargs = {'lw':1, 'color':'k'}

for harm in harmonic_numbers:
    ax.axvline(x=harm*f0, lw=0.25, color='r')#, ls='--', dashes=(2, 2))

ax.plot(fyy, pyy, **plot_kwargs)


xticks = np.arange(0, harmonic_numbers[-1]*f0+1, f0*5)
xticks_minor = np.arange(0, harmonic_numbers[-1]*f0+1, f0)
ax.set_xlim([xticks[0], xticks[-1]])
ax.set_ylim([0, 40])
ax.set_xticks(xticks)
ax.set_xticks(xticks_minor, minor=True)

ax.set_xlabel('Frequency (Hz)', fontsize=fontsize_labels)
ax.set_ylabel('dB/Hz SPL', fontsize=fontsize_labels)
ax.tick_params(axis='both', labelsize=fontsize_ticks)
ax.tick_params(axis='both', which='major', size=6)
ax.tick_params(axis='both', which='minor', size=4)

plt.tight_layout()
plt.show()
# fig.savefig('/om2/user/msaddler/pitchnet/assets_psychophysics/figures/tmp2.pdf', bbox_inches='tight')


In [ ]:
import os
import sys
import h5py
import glob
import json
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli

from stimuli_f0_labels import get_f0_bins, f0_to_label


hdf5_fn_regex = '/om2/user/msaddler/pitchnet/assets_datasets/tmp_HPv00.hdf5'
hdf5_fn_regex = '/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase03_filter_signalLPv02/*000000*.hdf5'

hdf5_fn_list = sorted(glob.glob(hdf5_fn_regex))
f0_values = []
for hdf5_fn in hdf5_fn_list:
    f = h5py.File(hdf5_fn, 'r')
    f0_values.extend(f['f0'][:].tolist())
    f.close()
    print(hdf5_fn, len(f0_values))


f = h5py.File(hdf5_fn, 'r')
hdf5_dataset_key_list = []
def get_dataset_paths(name, node):
    if isinstance(node, h5py.Dataset):
        hdf5_dataset_key_list.append(name)
f.visititems(get_dataset_paths)
# print('INPUT HDF5 FILE STRUCTURE:')
# for key in hdf5_dataset_key_list:
#     print('---|', key, f[key].shape, f[key].dtype)
#     if f[key].shape[0] == 1:
#         print('\t', f[key][0])


IDX = np.random.randint(len(f['f0'][:]))
f0 = f['f0'][IDX]
y = f['stimuli/signal_in_noise'][IDX]
snr = f['snr'][IDX]
dbspl = f['stimuli/signal_in_noise_dBSPL'][IDX]
fs = f['sr'][0]

if 'augmentation/signal_butterworth_bw' in f:
    print('BANDWIDTH=', f['augmentation/signal_butterworth_bw'][IDX])
if 'augmentation/signal_butterworth_fc' in f:
    print('FILTER FC=', f['augmentation/signal_butterworth_fc'][IDX])
    
f.close()


print(IDX)

ipd.display(ipd.Audio(y, rate=fs))
fyy, pyy = util_stimuli.power_spectrum(y, fs)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 2.5))
fontsize_labels = 16
fontsize_ticks = 12
plot_kwargs = {'lw':1, 'color':'k'}

harmonic_numbers = np.arange(f0, fs/2, f0) / f0
for harm in harmonic_numbers:
    ax.axvline(x=harm*f0, lw=0.25, color='r')#, ls='--', dashes=(2, 2))

ax.plot(fyy, pyy, **plot_kwargs)

xticks = np.arange(0, harmonic_numbers[-1]*f0+1, f0*5)
xticks_minor = np.arange(0, harmonic_numbers[-1]*f0+1, f0)
ax.set_xlim([xticks[0], xticks[-1]])
# ax.set_ylim([0, 40])
ax.set_xticks(xticks)
ax.set_xticks(xticks_minor, minor=True)

ax.set_xlabel('Frequency (Hz)', fontsize=fontsize_labels)
ax.set_ylabel('dB/Hz SPL', fontsize=fontsize_labels)
ax.tick_params(axis='both', labelsize=fontsize_ticks)
ax.tick_params(axis='both', which='major', size=6)
ax.tick_params(axis='both', which='minor', size=4)

title_str = 'f0={:.1f}, snr={:.1f}, dBSPL={:.1f}'.format(f0, snr, dbspl)
ax.set_title(title_str, fontsize=fontsize_ticks)

plt.tight_layout()
plt.show()



In [ ]:
f0_bins = get_f0_bins()
f0_bin_values = f0_to_label(f0_values, f0_bins)

uniq, coun = np.unique(f0_bin_values, return_counts=True)

plt.figure()
plt.plot(uniq, coun, '.')
plt.show()

In [ ]:
f0_bins.max()

In [ ]:
# Discretize the f0 values into bins
f0_bin_min=80
f0_bin_max=1e3
f0_min=80
f0_max=1e3
binwidth_in_octaves=1/192

f0_bins = get_f0_bins(f0_min=f0_bin_min, f0_max=f0_bin_max, binwidth_in_octaves=binwidth_in_octaves)

# Slightly hacky way to determine the correct value of f0_max to ensure all bins are equally wide
f0_min_label = np.squeeze(np.argwhere(f0_bins >= f0_min))[0]
f0_max_label = np.squeeze(np.argwhere(f0_bins < f0_max))[-1] + 1
num_bins = len(f0_bins)
if f0_max_label >= num_bins:
    expanded_f0_bins = get_f0_bins(f0_min=f0_bin_min,
                                   f0_max=2*f0_bin_max,
                                   binwidth_in_octaves=binwidth_in_octaves)
    expanded_f0_bins = expanded_f0_bins[0:num_bins+1]
    bin_adjusted_f0_min = f0_bins[f0_min_label]
    bin_adjusted_f0_max = expanded_f0_bins[f0_max_label]
else:
    bin_adjusted_f0_min = f0_bins[f0_min_label]
    bin_adjusted_f0_max = f0_bins[f0_max_label]

In [ ]:
bin_adjusted_f0_max

In [ ]:
import sys
import os
import json
import h5py
import glob
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

# import stimuli_compute_statistics
# import importlib
# importlib.reload(stimuli_compute_statistics)

# source_fn_regex = '/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/*.hdf5'
# output_fn = 'tmp_signal.json'
# running_freqs, running_mean_spectrum, running_n = stimuli_compute_statistics.serial_compute_mean_power_spectrum(
#     source_fn_regex, output_fn=output_fn, key_signal='/stimuli/signal')


In [ ]:
import h5py
f = h5py.File('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase0_filter_signalBPv00/PND_sr32000_BPv00.hdf5', 'r')
for v in f['stimuli'].values():
    print(v)
f.close()

In [ ]:
master_list = [
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/TMP_mean_spectrum_KEY_stimuli_noise.json', 'TLAS noise'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10_filter_signalBPv00/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08_BPv00'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10_filter_signalHPv00/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08_HPv00'),
    
# #     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/TMP_mean_spectrum_KEY_stimuli_noise.json', 'TLAS noise'),
# #     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08'),
# #     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10_filter_signalBPv01/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08_BPv01'),
# #     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10_filter_signalHPv01/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08_HPv01'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08spch/noise_TLAS_snr_neg10pos10/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08_spchOnly'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_v08inst/noise_TLAS_snr_neg10pos10/TMP_mean_spectrum_KEY_stimuli_signal.json', 'PNDv08_instOnly'),

#     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase03_filter_signalHPv02/TMP_mean_spectrum_KEY_stimuli_noise.json', 'synth_noise'),
#     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase03_filter_signalLPv02/TMP_mean_spectrum_KEY_stimuli_signal.json', 'synth_LPv02'),
#     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase03_filter_signalBPv02/TMP_mean_spectrum_KEY_stimuli_signal.json', 'synth_BPv02'),
#     ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase03_filter_signalHPv02/TMP_mean_spectrum_KEY_stimuli_signal.json', 'synth_HPv02'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase01_filter_signalLPv00/TMP_mean_spectrum_KEY_stimuli_signal.json', 'synth_LPv00'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase01_filter_signalBPv00/TMP_mean_spectrum_KEY_stimuli_signal.json', 'synth_BPv00'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase01_filter_signalHPv00/TMP_mean_spectrum_KEY_stimuli_signal.json', 'synth_HPv00'),
    ('/om/scratch/Fri/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase01_filter_signalHPv00/TMP_mean_spectrum_KEY_stimuli_noise.json', 'synth_noise'),
]

plt.figure(figsize=(12, 6))
plt.xscale('log')

for results_dict_fn, prolabel in master_list:
    print(results_dict_fn)
    with open(results_dict_fn, 'r') as rdf:
        results_dict = json.load(rdf)
        freqs = np.array(results_dict['freqs'])
        mean_spectrum = np.array(results_dict['mean_spectrum'])
        n = results_dict['n']
        key_signal = results_dict['key_signal']
        print(results_dict_fn)
    
    label = prolabel + ' (n={})'.format(n)
    IDX = freqs>0
    freqs = freqs[IDX]
    mean_spectrum = mean_spectrum[IDX]
    mean_spectrum = mean_spectrum - np.max(mean_spectrum)
    print(np.isfinite(mean_spectrum).all())
    plt.plot(freqs, mean_spectrum, label=label, lw=4)

plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('dB/Hz SPL')
plt.ylim([-50, 5])
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
# plt.plot(freqs[freqs>0], mean_power_spectrum_signal[freqs>0], 'k')
# plt.plot(freqs[freqs>0], mean_power_spectrum_noise[freqs>0], 'r')
plt.plot(running_freqs, running_mean_spectrum, 'k')
# plt.plot(freqs, mean_power_spectrum_noise, 'r')
plt.show()

In [ ]:
import sys
import os
import h5py
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures

import stimuli_generate_synthetic_tones_mcpherson
import importlib
importlib.reload(stimuli_generate_synthetic_tones_mcpherson)


In [ ]:
f0=200
fs=32000
dur=0.5
noise_filter_order = 6
noise_filter_cutoff = 6000
noise_filter_type = 'lowpass'
b, a = scipy.signal.butter(noise_filter_order,
                           np.array(noise_filter_cutoff)/(fs/2),
                           btype=noise_filter_type)
noise_filter = lambda x: scipy.signal.filtfilt(b, a, x)
y, y_params = stimuli_generate_synthetic_tones_mcpherson.mcpherson_noisy_tone(
    f0,
    fs,
    dur,
    dbsnr_component=-8,
    dbspl_overall=60.0,
    harmonic_numbers=range(1,11),
    kwargs_complex_tone={},
    kwargs_modified_uniform_masking_noise={},
    noise_filter=noise_filter)

fxx, pxx = util_stimuli.power_spectrum(y, fs)

fig, ax = plt.subplots(figsize=(10, 2))
util_figures.make_line_plot(ax, fxx, pxx, ylimits=[-30, 50])
plt.show()

print(y_params)
ipd.display(ipd.Audio(y, rate=fs))


In [ ]:
import sys
import os
import h5py
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures
import util_misc


# hdf5_filename = '/om/user/msaddler/data_pitchnet/mcpherson2020/testSPL_v01_f0min080_f0max320_dBSPLmin000_dBSPLmax120/testSPL_v01_f0min080_f0max320_dBSPLmin000_dBSPLmax120.hdf5'
hdf5_filename = '/om/user/msaddler/data_pitchnet/mcpherson2020/testSNR_v01_f0min080_f0max320_dBSPLmin060_dBSPLmax060/testSNR_v01_f0min080_f0max320_dBSPLmin060_dBSPLmax060.hdf5'

key_list = util_misc.get_hdf5_dataset_key_list(hdf5_filename)

f = h5py.File(hdf5_filename, 'r')

for k in key_list:
    print(k, f[k])
    

print(np.unique(f['snr_per_component']))
print(np.unique(f['dbspl']))

IDX = 0
y = f['tone_in_noise'][IDX]
fs = f['sr'][0]
fxx, pxx = util_stimuli.power_spectrum(y, fs)


fig, ax = plt.subplots(figsize=(10, 2))
util_figures.make_line_plot(ax, fxx, pxx, ylimits=[-30, 50])
plt.show()

for k in ['f0', 'snr', 'snr_per_component', 'dbspl', 'phase_mode']:
    print(k, f[k][IDX])
ipd.display(ipd.Audio(y, rate=fs))

f.close()


In [ ]:
import sys
import os
import h5py
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures
import util_misc

sys.path.append('/om2/user/msaddler/python-packages/bez2018model')
import bez2018model

f0 = 320
fs = 32000
dur = 0.5
snr = 12


low_harm = 20
num_harm = 22
harmonic_numbers = np.arange(low_harm, low_harm + num_harm)
print(harmonic_numbers)

kwargs_complex_tone={}
kwargs_modified_uniform_masking_noise={}
signal = util_stimuli.complex_tone(f0, fs, dur,
                                   harmonic_numbers=harmonic_numbers,
                                   amplitudes=None,
                                   **kwargs_complex_tone)
noise = util_stimuli.modified_uniform_masking_noise(fs, dur, **kwargs_modified_uniform_masking_noise)


y = util_stimuli.combine_signal_and_noise(signal, noise, snr)
y = util_stimuli.set_dBSPL(y, 120)

print(util_stimuli.get_dBSPL(y))


fig, ax = plt.subplots(figsize=(10, 2))
fxx, pxx = util_stimuli.power_spectrum(y, fs)
util_figures.make_line_plot(ax, fxx, pxx, ylimits=[np.min(pxx[50:]), None])
plt.show()

ipd.display(ipd.Audio(y, rate=fs))



In [ ]:
kwargs_nervegram_meanrates = {
    'meanrates_params': {
        'dur': 0.050,
        'fs':20e3,
        'buffer_start_dur': 0.07,
        'buffer_end_dur': 0.01,
    },
    'ANmodel_params': {
        'num_cfs': 100,
        'min_cf':125,
        'max_cf':14e3,
        'spont_list': [70],
        'cohc':1.0,
        'bandwidth_scale_factor':1.0,
        'IhcLowPass_cutoff': 3000.0,
        'IhcLowPass_order': 7,
    },
}
out_dict = bez2018model.nervegram_meanrates(y, fs, **kwargs_nervegram_meanrates)


In [ ]:
figsize=(6, 4)
nrows=2
ncols=3
gridspec_kw = {
    'wspace': 0.05,
    'hspace': 0.05,
    'width_ratios': [1, 8, 1],
    'height_ratios': [1, 5],
}
fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)

util_figures.make_stimulus_summary_plot(ax_arr,
                                        ax_idx_waveform=1,
                                        ax_idx_spectrum=3,
                                        ax_idx_nervegram=4,
                                        ax_idx_excitation=5,
                                        waveform=out_dict['signal'],
                                        nervegram=out_dict['meanrates'],
                                        sr_waveform=out_dict['signal_fs'],
                                        sr_nervegram=out_dict['meanrates_fs'],
                                        cfs=out_dict['cf_list'],
                                        tmin=None,
                                        tmax=None,
                                        treset=True,
                                        vmin=None,
                                        vmax=None,
                                        spines_to_hide_waveform=[],
                                        spines_to_hide_spectrum=[],
                                        spines_to_hide_excitation=[],
                                        nxticks=6,
                                        nyticks=6,
                                        kwargs_plot={},
                                        limits_buffer=0.1,
                                        ax_arr_clear_leftover=True)

plt.show()


In [ ]:
import sys
import os
import h5py
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures
import util_misc

sys.path.append('/om2/user/msaddler/python-packages/bez2018model')
import bez2018model

fn = '/om/user/msaddler/data_pitchnet/bernox2006/nharm10_f0min100_f0max300_TENlevel10dB_harmlevel20dBSPL/stim.hdf5'
fn = '/om/user/msaddler/data_pitchnet/bernox2006/nharm10_f0min100_f0max300_TENlevel40dB_harmlevel50dBSPL/stim.hdf5'
fn = '/om/user/msaddler/data_pitchnet/bernox2006/nharm10_f0min100_f0max300_TENlevel65dB_harmlevel75dBSPL/stim.hdf5'
f = h5py.File(fn, 'r')

fs = f['sr'][0]

IDX = np.random.randint(f['tone_in_noise'].shape[0])
y = f['tone_in_noise'][IDX]

fig, ax = plt.subplots(figsize=(10, 2))
fxx, pxx = util_stimuli.power_spectrum(y, fs)
util_figures.make_line_plot(ax, fxx, pxx, ylimits=[np.min(pxx[50:]), None])
plt.show()

print(util_stimuli.get_dBSPL(y))
print(util_stimuli.get_bandlimited_power(y, fs, band=[935, 1068.1], dBSPL=True))

ipd.display(ipd.Audio(y, rate=fs))

f.close()



In [ ]:
kwargs_nervegram_meanrates = {
    'meanrates_params': {
        'dur': 0.050,
        'fs':20e3,
        'buffer_start_dur': 0.07,
        'buffer_end_dur': 0.01,
    },
    'ANmodel_params': {
        'num_cfs': 100,
        'min_cf':125,
        'max_cf':14e3,
        'spont_list': [70],
        'cohc':1.0,
        'bandwidth_scale_factor':1.0,
        'IhcLowPass_cutoff': 3000.0,
        'IhcLowPass_order': 7,
    },
}
out_dict = bez2018model.nervegram_meanrates(y, fs, **kwargs_nervegram_meanrates)


In [ ]:
figsize=(6, 4)
nrows=2
ncols=3
gridspec_kw = {
    'wspace': 0.05,
    'hspace': 0.05,
    'width_ratios': [1, 8, 1],
    'height_ratios': [1, 5],
}
fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)

util_figures.make_stimulus_summary_plot(ax_arr,
                                        ax_idx_waveform=1,
                                        ax_idx_spectrum=3,
                                        ax_idx_nervegram=4,
                                        ax_idx_excitation=5,
                                        waveform=out_dict['signal'],
                                        nervegram=out_dict['meanrates'],
                                        sr_waveform=out_dict['signal_fs'],
                                        sr_nervegram=out_dict['meanrates_fs'],
                                        cfs=out_dict['cf_list'],
                                        tmin=None,
                                        tmax=None,
                                        treset=True,
                                        vmin=None,
                                        vmax=None,
                                        spines_to_hide_waveform=[],
                                        spines_to_hide_spectrum=[],
                                        spines_to_hide_excitation=[],
                                        nxticks=6,
                                        nyticks=6,
                                        kwargs_plot={},
                                        limits_buffer=0.1,
                                        ax_arr_clear_leftover=True)

plt.show()


In [ ]:
fn = '/om/scratch/Mon/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/sr2000_cf1000_species002_spont070_BW10eN1_IHC0050Hz_IHC7order/bez2018meanrates_066_000000-003500.hdf5'
f = h5py.File(fn, 'r')

# for _ in util_misc.get_hdf5_dataset_key_list(fn):
#     print(f[_])

IDX = np.random.randint(f['meanrates'].shape[0])
nervegram = f['meanrates'][IDX]
sr_nervegram = f['meanrates_fs'][0]
IDX = np.random.randint(f['meanrates'].shape[0])
waveform = f['signal'][IDX]
sr_waveform = f['signal_fs'][0]
cfs = f['cf_list'][0]


figsize=(8, 12)
nrows=2
ncols=3
gridspec_kw = {
    'wspace': 0.05,
    'hspace': 0.05,
    'width_ratios': [1, 8, 1],
    'height_ratios': [1, 5],
}
fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)

util_figures.make_stimulus_summary_plot(ax_arr,
                                        ax_idx_waveform=1,
                                        ax_idx_spectrum=3,
                                        ax_idx_nervegram=4,
                                        ax_idx_excitation=5,
                                        waveform=waveform,
                                        nervegram=nervegram,
                                        sr_waveform=sr_waveform,
                                        sr_nervegram=sr_nervegram,
                                        cfs=cfs,
                                        tmin=None,
                                        tmax=None,
                                        treset=True,
                                        vmin=None,
                                        vmax=None,
                                        spines_to_hide_waveform=[],
                                        spines_to_hide_spectrum=[],
                                        spines_to_hide_excitation=[],
                                        nxticks=6,
                                        nyticks=6,
                                        kwargs_plot={},
                                        limits_buffer=0.1,
                                        ax_arr_clear_leftover=True)


f.close()


In [ ]:
import sys
import os
import h5py
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures
import util_misc

sys.path.append('/om2/user/msaddler/python-packages/bez2018model')
import bez2018model

import pitchrepnet_stimuli_generate_BernsteinOxenhamFixedFilter as psg
import importlib
importlib.reload(psg)

hdf5_filename = 'tmp.hdf5'
psg.generate_fixed_bandpass_filter_dataset(hdf5_filename,
                                           fs=20e3,
                                           dur=2e0,
                                           phase_modes=['sine', 'rand'],
                                           jitter_modes=[None],
                                           bandpass_fl_min=6e1,
                                           bandpass_fl_max=6e3,
                                           bandpass_fl_num=500,
                                           bandpass_fl_spacing='linear',
                                           bandpass_filter_bw=2e3,
                                           bandpass_filter_order=4,
                                           f0_min=80.0,
                                           f0_max=640.0,
                                           f0_step=1.005,
                                           threshold_dBSPL=33.3,
                                           component_dBSL=15.0,
                                           noise_dBHzSPL=15.0,
                                           noise_attenuation_start=600.0,
                                           noise_attenuation_slope=2,
                                           random_seed=858,
                                           disp_step=5)


In [ ]:
hdf5_filename = '/om/scratch/Mon/msaddler/data_hearinglossnet/pitchrepnet_train2afc_v00/stim.hdf5'
hdf5_filename = 'tmp.hdf5'

f = h5py.File(hdf5_filename, 'r')
for k in util_misc.get_hdf5_dataset_key_list(f):
    print(k, f[k].dtype, f[k].shape)
    if len(f[k].shape) < 2:
        print('|___ unique values:', np.unique(f[k][:]))

plt.figure(figsize=(12, 2))
for _ in range(2):
    IDX = np.random.randint(low=0, high=f['f0'].shape[0])
    print(IDX)

    fs = f['config_tone/fs'][0]
    f0 = f['f0'][IDX]
    x = f['tone_in_noise'][IDX]
    for k in ['f0', 'low_harm', 'bandpass_fl']:
        print('... {} = {}'.format(k, f[k][IDX]))
    fxx, pxx = util_stimuli.power_spectrum(x, fs)

    plt.plot(fxx/f0, pxx)

    ipd.display(ipd.Audio(x, rate=fs))

plt.ylim([-20, None])
# plt.xlim([0, 30])
plt.show()
    
f.close()


In [ ]:
fs = 32e3

frequency_response_in_dB = psg.get_bandpass_filter_frequency_response(1500, 3500, fs=fs)

inharmonic_jitter_pattern = 0.000*np.random.uniform(low=-0.5, high=0.5, size=[1000])
x, x_metadata = psg.harmonic_or_inharmonic_complex_tone(f0=2000.0,
                                                        fs=fs,
                                                        dur=0.5,
                                                        inharmonic_jitter_pattern=inharmonic_jitter_pattern,
                                                        frequency_response_in_dB=frequency_response_in_dB)

inharmonic_jitter_pattern = 1.0*np.random.uniform(low=-0.5, high=0.5, size=[1000])
inharmonic_jitter_pattern[0] = 0
y, y_metadata = psg.harmonic_or_inharmonic_complex_tone(f0=2000.0,
                                                        fs=fs,
                                                        dur=0.5,
                                                        inharmonic_jitter_pattern=inharmonic_jitter_pattern,
                                                        frequency_response_in_dB=frequency_response_in_dB)

fxx, pxx = util_stimuli.power_spectrum(x, fs)
fyy, pyy = util_stimuli.power_spectrum(y, fs)

plt.figure()
plt.plot(fxx, pxx)
plt.plot(fyy, pyy)
plt.ylim([pyy.min(), None])
plt.show()

ipd.display(ipd.Audio(x, rate=fs))
ipd.display(ipd.Audio(y, rate=fs))

for k in x_metadata.keys():
    print(k, x_metadata[k], y_metadata[k])


In [ ]:
fs = 20e3
baseline_fl = 2.5e3
baseline_fh = 3.5e3
order = 4
baseline_frequency_response_in_dB = psg.get_bandpass_filter_frequency_response(
    baseline_fl,
    baseline_fh,
    fs=fs,
    order=order)


desired_fl = np.random.randint(8e1, 8e3)
print(desired_fl)
desired_fl_gain_in_dB = -15
frequency_response_in_dB = psg.shift_bandpass_filter_frequency_response(
    desired_fl,
    desired_fl_gain_in_dB,
    fs=fs,
    unshifted_passband=None,
    frequency_response_in_dB=baseline_frequency_response_in_dB)


plt.figure()
for f0 in [160, 200]:
    x, x_metadata = psg.harmonic_or_inharmonic_complex_tone(
        f0=f0,
        fs=fs,
        dur=0.5,
        inharmonic_jitter_pattern=0.0 * np.random.randn(1000) + 1,
        frequency_response_in_dB=frequency_response_in_dB)
    ipd.display(ipd.Audio(x, rate=fs))
    
    fxx, pxx = util_stimuli.power_spectrum(x, fs)
    plt.plot(fxx, pxx)    

plt.ylim([33.0-10, None])
plt.show()


In [ ]:
get_jitter_pattern = lambda: 5
get_jitter_pattern()


In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures
import util_misc

fn_regex = '/om/scratch/Mon/msaddler/data_pitchnet/PND_synthetic/noise_UMNm_snr_neg10pos10_phase0_filter_signalLPv00/*.hdf5'
fn = sorted(glob.glob(fn_regex))[0]

for k in util_misc.get_hdf5_dataset_key_list(fn):
    print(k)


f = h5py.File(fn, 'r')

fc = f['augmentation/signal_butterworth_fc'][:]

print(fc.shape, fc.min(), fc.max(), fc.mean())

fc_range = [1e3, 1e4]
fc_new = np.exp(np.random.uniform(np.log(fc_range[0]), np.log(fc_range[1]), size=[700000]))

print(fc_new.shape, fc_new.min(), fc_new.max(), fc_new.mean())

f.close()



In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_figures
import util_misc

# fn = '/om/user/msaddler/data_pitchnet/shackcarl1994/AltPhase_v01_f0min080_f0max320/AltPhase_v01_f0min080_f0max320.hdf5'
fn = '/om/user/msaddler/data_pitchnet/mooremoore2003/MooreMoore2003_frequencyShiftedComplexes_f0_080to480Hz/MooreMoore2003_frequencyShiftedComplexes_f0_080to480Hz.hdf5'
fn = '/om/user/msaddler/data_pitchnet/moore1985/Moore1985_MistunedHarmonics_v01/Moore1985_MistunedHarmonics_v01.hdf5'
fn = '/om/user/msaddler/data_pitchnet/oxenham2004/Oxenham2004_transposedTones_f0_080to320Hz/Oxenham2004_transposedTones_f0_080to320Hz.hdf5'
fn = '/om/user/msaddler/data_pitchnet/mcpherson2020/testSPL_v01_f0min080_f0max320_dBSPLmin000_dBSPLmax120/testSPL_v01_f0min080_f0max320_dBSPLmin000_dBSPLmax120.hdf5'
fn = '/om/user/msaddler/data_pitchnet/mcpherson2020/testSNR_v01_f0min080_f0max320_dBSPLmin060_dBSPLmax060/testSNR_v01_f0min080_f0max320_dBSPLmin060_dBSPLmax060.hdf5'
fn = '/om/user/msaddler/data_pitchnet/bernox2005/FixedFilter_f0min100_f0max300/FixedFilter_f0min100_f0max300.hdf5'
f = h5py.File(fn, 'r')

f0_vals = np.unique(f['f0'][:])
# print(np.unique(f0_vals[1:] / f0_vals[0:-1]))
print(f0_vals.shape)
print(f0_vals.min(), f0_vals.max())
f0_vals = f['f0'][:]
print(f0_vals.shape)

# print(np.unique(f['snr_per_component'][:]))

for f0 in [125.0, 250.0]:
    IDX = np.logical_and(f0_vals >= f0 * 0.96, f0_vals <= f0*1.04)
    print(f0, IDX.sum())

f.close()

In [ ]:
534/3

In [ ]:
f0_ref_min=90.0
f0_ref_max=300.0
f0_ref_n_step=5
f0_ref_list = np.power(2, np.linspace(np.log2(f0_ref_min), np.log2(f0_ref_max), f0_ref_n_step))
for f0_ref in f0_ref_list:    
    IDX = np.logical_and(f0_vals >= f0_ref*0.94, f0_vals <= f0_ref*1.06)
    print(f0_ref, IDX.sum())


In [ ]:
90 * 1.06

In [ ]:
533 * 532 / 2

In [ ]:
import sys
import os
import json
import glob
import h5py

import numpy as np
import scipy.signal
import scipy.io.wavfile
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import importlib

sys.path.append('/om2/user/msaddler/python-packages/bez2018model')
import bez2018model
importlib.reload(bez2018model)

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_figures
importlib.reload(util_figures)
import util_stimuli
importlib.reload(util_stimuli)
import util_misc
importlib.reload(util_misc)

import stimuli_generate_TransposedTones
importlib.reload(stimuli_generate_TransposedTones)

hdf5_filename='/om/user/msaddler/data_pitchnet/oxenham2004/Oxenham2004_transposedTones_f0_080to320Hz_v01/stim.hdf5'
# stimuli_generate_TransposedTones.generate_Oxenham2004_dataset(hdf5_filename, step_size_in_octaves=1/(12*16*16*2))


In [ ]:
f = h5py.File(hdf5_filename, 'r')


f0_vals = f['f0'][:]

print(np.unique(f0_vals).shape)



IDX = np.random.randint(24576)
IDX = 8140
fs = f['sr'][0]
x = f['stimuli/signal_in_noise'][IDX]
f0 = f['f0'][IDX]
print(f0)

plt.figure()
fxx, pxx = util_stimuli.power_spectrum(x, fs)
plt.plot(fxx, pxx)
plt.show()

f.close()

ipd.display(ipd.Audio(x, rate=fs))


In [ ]:
x = f0_vals[f0_vals <= 80*(1+100/6/100)]
np.unique(x).shape


In [ ]:
kwargs_nervegram_meanrates = {
    'meanrates_params': {
        'dur': 0.050,
        'fs': 20e3,
        'buffer_start_dur': 0.07,
        'buffer_end_dur': 0.01,
    },
    'ANmodel_params': {
        'num_cfs': 100,
        'min_cf': 125,
        'max_cf': 14e3,
        'spont_list': [70.0],
        'cohc': 1.0,
        'bandwidth_scale_factor': 1.0,
        'IhcLowPass_cutoff': 3000,
        'IhcLowPass_order': 7,
    },
}
out_dict = bez2018model.nervegram_meanrates(x, fs, **kwargs_nervegram_meanrates)



figsize=(6, 3.5)
nrows=2
ncols=3
gridspec_kw = {
    'wspace': 0.15,
    'hspace': 0.15,
    'width_ratios': [1, 6, 1],
    'height_ratios': [1, 4],
}
fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, gridspec_kw=gridspec_kw)
util_figures.make_stimulus_summary_plot(ax_arr,
                                        ax_idx_waveform=1,
                                        ax_idx_spectrum=3,
                                        ax_idx_nervegram=4,
                                        ax_idx_excitation=5,
                                        waveform=out_dict['signal'],
                                        nervegram=out_dict['meanrates'],
                                        sr_waveform=out_dict['signal_fs'],
                                        sr_nervegram=out_dict['meanrates_fs'],
                                        cfs=out_dict['cf_list'],
                                        tmin=None,
                                        tmax=None,
                                        treset=True,
                                        vmin=None,
                                        vmax=None,
                                        spines_to_hide_waveform=[],
                                        spines_to_hide_spectrum=[],
                                        spines_to_hide_excitation=[],
                                        nxticks=6,
                                        nyticks=6,
                                        kwargs_plot={},
                                        limits_buffer=0.2,
                                        ax_arr_clear_leftover=True)

xticks = [0, 640]
ax_arr[1,2].set_xlim(xticks)
ax_arr[1,2].set_xticks(xticks)
ax_arr[1,2].set_xticklabels(xticks)

plt.show()


In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_misc
import util_figures


regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PND_sr32000_v08_*.hdf5'

list_fn = sorted(glob.glob(regex_fn))
fn = list_fn[0]

# util_misc.get_hdf5_dataset_key_list(fn)

with h5py.File(fn, 'r') as f:
    sr = f['sr'][0]
    IDX = np.random.randint(0, f['nopad_f0_mean'].shape[0])
    f0 = f['nopad_f0_mean'][IDX]
    y_fg = util_stimuli.set_dBSPL(f['stimuli/signal'][IDX], 60.0)
    y_bg = util_stimuli.set_dBSPL(f['stimuli/noise'][IDX], 60.0)


fxx, pxx = util_stimuli.power_spectrum(y_fg, sr)

fenv, penv = util_stimuli.get_spectral_envelope_lp(y_fg, sr, M=12)
penv = penv - penv.max() + pxx.max()

print(util_stimuli.get_dBSPL(y_fg))
print(util_stimuli.get_dBSPL(y_bg))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 2.5))
ax.plot(fxx, pxx, color='k', lw=1.0)
ax.plot(fenv, penv, color='r', lw=1.0)
ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, sr/2],
                              ylimits=None,
                              spines_to_hide=['right', 'top'])
plt.show()

ipd.display(ipd.Audio(y_fg, rate=sr))


In [ ]:
y = y_fg

t = np.arange(0, len(y)) / sr
x = np.zeros_like(y)
for f in np.arange(f0, sr/2, f0):
    x = x + np.sin(2*np.pi*f*t)
# x = np.random.randn(*t.shape)

b_lp, a_lp = util_stimuli.get_spectral_envelope_lp_coefficients(y, M=12)
x = scipy.signal.lfilter(b_lp, a_lp, x)


x = util_stimuli.set_dBSPL(x, 60.0)

fyy, pyy = util_stimuli.power_spectrum(y, sr)
fxx, pxx = util_stimuli.power_spectrum(x, sr)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 2.5))
ax.plot(fyy, pyy, color='k', lw=1.0)
ax.plot(fxx, pxx, color='r', lw=1.0)

ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, sr/2],
                              ylimits=None,
                              spines_to_hide=['right', 'top'])
plt.show()

ipd.display(ipd.Audio(y, rate=sr))
ipd.display(ipd.Audio(x, rate=sr))

plt.figure()
plt.plot(t, y, color='k')
plt.plot(t, x, color='r')
plt.show()


In [ ]:
def get_spectral_envelope(x, fs, period=1/1000):
    '''
    Computes spectral envelope of given signal via "Cepstral Windowing"
    as described here:
    https://ccrma.stanford.edu/~jos/sasp/Cepstral_Windowing.html
    
    Args
    ---
    x (np.ndarray): input waveform (Pa)
    fs (int): sampling rate (Hz)
    period (float): period used to set cutoff for lowpass filter (s)
    '''
    freqs = np.fft.rfftfreq(len(x), d=1/fs)
    n_cutoff = min(int(period * sr), len(x)-1)
    lifter = np.zeros_like(x)
    lifter[:n_cutoff] = 1
    lifter[n_cutoff] = 1/2
    log_magnitude_spectrum = np.log(np.abs(np.fft.rfft(x)))
#     log_magnitude_spectrum = 10 * np.log10((2 * np.square(np.abs(np.fft.rfft(x) / len(x)))) / np.square(20e-6))
    real_cepstrum = np.fft.irfft(log_magnitude_spectrum).real
    spectral_envelope = np.fft.rfft(lifter * real_cepstrum).real
    return freqs, spectral_envelope, log_magnitude_spectrum


y = y_fg
print(y.shape)
freqs, spectral_envelope, log_magnitude_spectrum = get_spectral_envelope(y, sr)


log_magnitude_spectrum = log_magnitude_spectrum + spectral_envelope.max() - log_magnitude_spectrum.max()

# spectral_envelope = spectral_envelope * np.max(log_magnitude_spectrum) / np.max(spectral_envelope)

# spectral_envelope = spectral_envelope + 25

plt.figure()
plt.plot(freqs, log_magnitude_spectrum)
plt.plot(freqs, spectral_envelope)
# plt.ylim([-5, None])
plt.show()


In [ ]:
import stimuli_compute_statistics
import importlib
importlib.reload(stimuli_compute_statistics)

import glob

regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PND_sr32000_v08_*.hdf5'
list_fn = sorted(glob.glob(regex_fn))
fn = list_fn[0]

fn_output = 'tmp.hdf5'


stimuli_compute_statistics.compute_spectral_statistics(fn, fn_output)


In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_stimuli
import util_misc
import util_figures


with h5py.File('tmp.hdf5', 'r') as f:

    for k in util_misc.get_hdf5_dataset_key_list(f):
        print(k)
        
    IDX = np.random.randint(0, 13600)
    key = 'stimuli/signal'
    fs = f['sr'][0]
    fxx = f['freqs'][0]
    pxx = f[key + '_power_spectrum'][IDX]
    b_lp = f[key + '_spectral_envelope_b_lp'][IDX]
    a_lp = f[key + '_spectral_envelope_a_lp'][IDX]
    
    a_values = f[key + '_spectral_envelope_a_lp'][0:13600]


freqs, h = scipy.signal.freqz(b_lp, a_lp, len(pxx), fs=fs)
lp_spectral_envelope = 20 * np.log10(np.abs(h))

lp_spectral_envelope = lp_spectral_envelope - lp_spectral_envelope.max() + pxx.max()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 2.5))
ax.plot(fxx, pxx, color='k', lw=1.0)
ax.plot(freqs, lp_spectral_envelope, color='r', lw=1.0)

# Use this function to specify axis scaling, limits, labels, etc.
ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, fs/2],
                              ylimits=None,
                              spines_to_hide=['right', 'top'])
plt.show()



In [ ]:
plt.figure()
for i in range(a_values.shape[1]):
    plt.hist(a_values[:, i], 100)
plt.xlim([-5, 5])
plt.show()
